In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor

# 1. Load and preprocess data
df = pd.read_csv(r'insurance.csv')
df['Height_m'] = df['Height'] / 100
df['BMI']      = df['Weight'] / (df['Height_m'] ** 2)
df.drop(columns=['Height', 'Weight', 'Height_m'], inplace=True)

# 2. Separate features and target
X = df.drop(columns=['PremiumPrice'])
y = df['PremiumPrice']

# 3. Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 5. Build and train the neural network
mlp = MLPRegressor(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    max_iter=500,
    random_state=42
)
mlp.fit(X_train, y_train)

# 6. Evaluate on test set
y_pred = mlp.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("Neural Network Performance:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE : {mae:.2f}")
print(f"  R²  : {r2:.4f}")


Neural Network Performance:
  RMSE: 4530.55
  MAE : 3487.73
  R²  : 0.5187


c:\Users\Shubhashi\anaconda3\envs\comp_vision\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Hyperparameter tuning 

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
from scipy.stats import randint, uniform

# 1. Load & preprocess data
df = pd.read_csv(r'insurance.csv')
df['Height_m'] = df['Height'] / 100
df['BMI']      = df['Weight'] / (df['Height_m'] ** 2)
df.drop(columns=['Height', 'Weight', 'Height_m'], inplace=True)

X = df.drop(columns=['PremiumPrice'])
y = df['PremiumPrice']

# 2. Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 4. Define the MLP and parameter distributions
mlp = MLPRegressor(random_state=42, early_stopping=True, validation_fraction=0.1)

param_dist = {
    'hidden_layer_sizes': [(50,),(100,),(64,32),(128,64,32)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': uniform(1e-5, 1e-3),
    'learning_rate_init': uniform(1e-4, 1e-2),
    'max_iter': [200, 500, 1000]
}

# 5. Setup RandomizedSearchCV
cv = KFold(n_splits=5, shuffle=True, random_state=42)
search = RandomizedSearchCV(
    mlp,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=cv,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

# 6. Run search
search.fit(X_train, y_train)

print("Best parameters:", search.best_params_)
print("Best CV R²:", search.best_score_)

# 7. Evaluate best estimator on test set
best_mlp = search.best_estimator_
y_pred = best_mlp.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("\nTuned MLP Performance on Test Set:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE : {mae:.2f}")
print(f"  R²  : {r2:.4f}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters: {'activation': 'relu', 'alpha': np.float64(0.0008600385777897993), 'hidden_layer_sizes': (128, 64, 32), 'learning_rate_init': np.float64(0.0034089802485264917), 'max_iter': 1000, 'solver': 'adam'}
Best CV R²: 0.5617372107092837

Tuned MLP Performance on Test Set:
  RMSE: 3558.44
  MAE : 2717.39
  R²  : 0.7031


Here’s a consolidated comparison including your newly tuned MLP against the benchmarks from before:

| Model                              |  CV R² | Test R² | Test RMSE | Test MAE |
| ---------------------------------- | -----: | ------: | --------: | -------: |
| **Linear Regression (original y)** |  0.617 |  0.7136 |   3,494.4 |  2,586.2 |
| **Decision Tree (default)**        |  0.500 |  0.7870 |   3,013.4 |  1,333.3 |
| **Decision Tree (tuned)**          |  0.668 |  0.8220 |   2,755.1 |  1,757.3 |
| **Random Forest (untuned)**        |  0.721 |  0.8625 |   2,421.4 |  1,439.5 |
| **Gradient Boosting (untuned)**    |  0.735 |  0.8549 |   2,487.1 |  1,675.1 |
| **Random Forest (tuned)**          | 0.7096 |  0.8289 |   2,700.9 |  1,846.4 |
| **Gradient Boosting (tuned)**      | 0.7083 |  0.8393 |   2,617.6 |  1,845.4 |
| **MLPRegressor (tuned)**           | 0.5617 |  0.7031 |   3,558.4 |  2,717.4 |

---

### 🔍 What This Tells Us

* The **tuned MLP** falls **below** the **linear regression baseline** in both CV R² (0.5617 vs 0.617) and Test R² (0.7031 vs 0.7136), and has **higher errors** (RMSE/MAE).
* All **tree‐based ensembles** (RF, GBM) **outperform** both the MLP and the linear model, with **Random Forest (untuned)** remaining the top performer.
* The **tuned Decision Tree** improved over its default version but still can’t beat the ensembles.
* Even after hyperparameter search, the neural network doesn’t capture this tabular data’s patterns as effectively as tree methods.

---

### 🚀 Recommendation

Stick with **Random Forest** (or further tune GBM) for production pricing, and reserve the MLP approach for scenarios where you expect very **highly nonlinear feature interactions** or have **much larger** datasets.
